#Resnet50 Training


In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp /content/drive/MyDrive/dataset.zip /content/


In [6]:
!unzip -q dataset.zip


In [7]:
# 1. Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Higher res for ResNet50
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])   # ImageNet std
])

In [8]:
# 2. Dataset
dataset = datasets.ImageFolder(root="/content/dataset", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

In [9]:
# 3. Model
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


In [10]:

# 4. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
# 5. Training Loop
for epoch in range(5):  # Start with 5, increase if needed
    model.train()
    total_loss = 0
    correct = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = correct / len(dataset)
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}, Accuracy={accuracy:.4f}")

Epoch 1: Loss=281.8349, Accuracy=0.9390
Epoch 2: Loss=83.7206, Accuracy=0.9844
Epoch 3: Loss=58.8618, Accuracy=0.9901
Epoch 4: Loss=47.6201, Accuracy=0.9917
Epoch 5: Loss=40.1688, Accuracy=0.9929


In [12]:
# 6. Save Model
torch.save(model.state_dict(), "resnet50_deepfake_frame.pth")
print("Model saved.")

Model saved.


In [13]:
from google.colab import files
files.download("resnet50_deepfake_frame.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>